In [1]:
import requests
import json
import pandas as pd
import time
import telegram 

headers= {
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
      } 



In [18]:
def crawl(url):
  res = requests.get(url, headers=headers)

  #크롤링한 데이터를 Json형태로 변환
  data = json.loads(res.text)
  items = data['result']

  #변환된 Json데이터를 DataFrame화
  df = pd.DataFrame.from_dict(items)
  df = df.rename(columns={'hscpNm':'단지명','dealPrcMin':'매매최저가','dealPrcMax':'매매최고가',
                        'leasePrcMin':'전세최저가','leasePrcMax':'전세최고가','useAprvYmd':'승인날짜'})

  #불필요한 문자들 정규식으로 삭제
  df["매매최저가"].replace("[a-z<='_>/]", "", regex=True, inplace = True) 
  df["매매최고가"].replace("[a-z<='_>/]", "", regex=True, inplace = True)
  df["전세최저가"].replace("[a-z<='_>/]", "", regex=True, inplace = True) 
  df["전세최고가"].replace("[a-z<='_>/]", "", regex=True, inplace = True)

  #원하는 DataFrame 컬럼을 뽑아온뒤 지역엑셀 파일로 저장
  P_list = df[['단지명','전세최저가','전세최고가','승인날짜','매매최저가','매매최고가']]

  return P_list


In [20]:
#은평구 수색동
url_susaek = "https://m.land.naver.com/cluster/ajax/complexList?rletTpCd=APT&tradTpCd=A1&z=15&lat=37.5867032&lon=126.8957615&btm=37.5771805&lft=126.8800116&top=37.5962247&rgt=126.9115114&showR0=&totCnt=8&cortarNo=1138010100&isOnlyIsale=false "
df = crawl(url_susaek)
df.to_excel('./Legion/수색.xlsx')

In [23]:
#영등포구 문래동6가
url_mulae6 = "https://m.land.naver.com/cluster/ajax/complexList?rletTpCd=APT&tradTpCd=A1&z=16&lat=37.5213424&lon=126.8838777&btm=37.516577&lft=126.8760028&top=37.5261074&rgt=126.8917527&showR0=&totCnt=10&cortarNo=1156012400&isOnlyIsale=false"
df =crawl(url_mulae6)
df.to_excel('./Legion/문래6가.xlsx')

In [24]:
#영등포구 여의도동
url_yeido = "https://m.land.naver.com/cluster/ajax/complexList?itemId=2122100121&mapKey=&lgeo=2122100121&rletTpCd=APT&tradTpCd=A1&z=14&lat=37.5304093&lon=126.9357179&btm=37.5113483&lft=126.904218&top=37.5494655&rgt=126.9672177&cortarNo=1156011000&isOnlyIsale=false"
df = crawl(url_yeido)
df.to_excel('./Legion/여의도.xlsx')

In [25]:
#강남구 대치동
url_daech = "https://m.land.naver.com/cluster/ajax/complexList?rletTpCd=APT&tradTpCd=A1&z=15&lat=37.4996377&lon=127.057867&btm=37.4902571&lft=127.0420741&top=37.5090171&rgt=127.0736598&showR0=&totCnt=66&cortarNo=1168010600"
df = crawl(url_daech)
df.to_excel('./Legion/대치.xlsx')

In [39]:
telegram_config = {}
# config 파일 읽기
with open('./Mod/telegram_config', 'r') as f:
    # 모든 줄 읽어오기
    configs = f.readlines()
    # 한 줄씩 확인해서
    for config in configs:
        # 줄바꿈기호 제거(\n) 후 =로 문자열 분리
        # key, value로 언패킹 (2개 나올것이 확실하기 때문에)
        key, value = config.rstrip().split('=')
        # config 딕셔너리에 키-값 추가
        telegram_config[key] = value

token = telegram_config['token']


In [40]:
bot = telegram.Bot(token)

In [41]:
updates = bot.get_updates()

In [42]:
chat_id = updates[-1].message.chat.id

In [46]:
last_id = updates[-1].update_id

In [45]:
# 서버에서 텔레그램 메시지 확인, 응답 보내기
bot.send_message(chat_id, '아파트 단지 시세조회 시작\n수색/문래6가/여의도/대치 중에 한곳을 입력하세요\n예>대치')
while True:
    
    try:
        # 신규 메시지가 없을 경우 에러가 발생 
        # list index out of range
        # 따라서, try - except 문으로 묶어줌
        
        new_message = bot.get_updates(offset=last_id)[-1]

        if new_message.message.text == '대치':
            # 관련 메시지 발송
            bot.send_document(chat_id, open('/./gitsmw/Agora/Legion/대치.xlsx', 'rb'))
        
        elif new_message.message.text == '수색':
            bot.send_document(chat_id, open('/./gitsmw/Agora/Legion/수색.xlsx', 'rb'))

        elif new_message.message.text == '문래6가':
            bot.send_document(chat_id, open('/./gitsmw/Agora/Legion/문래6가.xlsx', 'rb'))

        elif new_message.message.text == '여의도':
            bot.send_document(chat_id, open('/./gitsmw/Agora/Legion/여의도.xlsx', 'rb'))

        # offset 값 최신화 (update_id) + 1 해줘서 그 다음부터 메시지부터 확인하도록
        last_id = new_message.update_id + 1
    except:
        pass

    # 텔레그램 서버 부하 줄이기 위해 3초마다 확인
    time.sleep(3)





KeyboardInterrupt: 